In [26]:
using DataFrames, CSV, Impute, Dates, Plots, Statistics, Interpolations

In [33]:
include("utils.jl");

## Choose data file

In [28]:
data_available = [
    "boston_09_12",
    "boston_13_15",
    "boston_16_17",
    "boston_18_20",
    "hanscom_18_20",
    "pittsfield_18_20"
];
choice = "boston_18_20";

In [29]:
df = CSV.File(string("data/", choice, ".csv")) |> DataFrame;

In [30]:
dfdaily = filter(row -> strip(row[:REPORT_TYPE]) == "SOD", df);
dfh = filter(row -> !(strip(row[:REPORT_TYPE]) in ["SOD", "SOM"]), df)
sum(ismissing.(dfh[:, :HourlyDryBulbTemperature]))

0

In [31]:
factors =  [(:HourlyDryBulbTemperature,:temp), (:HourlyPrecipitation,:prec), (:HourlySeaLevelPressure,:pres)];

In [32]:
mdata = selectdata(dfh, factors);
first(mdata, 4)

,sampleT,temp,prec,pres
,DateTime,Float64,Float64,Float64
1,2018-01-01T01:00:00,2.0,0.0,30.2
2,2018-01-01T02:00:00,1.0,0.0,30.21
3,2018-01-01T03:00:00,1.0,0.0,30.21
4,2018-01-01T04:00:00,1.0,0.0,30.2


## Define characteristic of training set

In [39]:
start = 2018, 1, 2
dayslookback = 5
ndays = 360
predicttype = :temp
offset = 1;

In [42]:
X, y = build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=offset);

In [43]:
(train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y);

In [ ]:
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
        show_progress=false
    ),
#     max_depth=1:5,
#     minbucket=10:10:100
    max_depth = 1:5,
    minbucket = 10
    show_progress=false # <-- uncomment to avoid having all the progress bars show up
)
IAI.fit!(grid, train_X, train_y)

In [ ]:
lnr = IAI.get_learner(grid)
IAI.score(lnr, test_X, test_y)  # Testing on the same year